# Overview

The goal of this project is to train a model using Reuters dataset in English and then extend it to German and French using https://github.com/facebookresearch/MUSE

# Dataset

The Reuters Corpus contains 10,788 news documents totaling 1.3 million words. The documents have been classified into 90 topics, and grouped into two sets, called "training" (a set of 7769 documents) and "test" (a set of 3019 documents). The Reuters Corpus can be found in the Python's NLTK library. This dataset contains news articles which can cover different topics, thus are assigned to multiple labels. 


# Document Cleaning and Representation

Before applying any machine learning techniques, the text should be prepared first, in other words, document representation should be made available. The process of converting data to something a computer can understand is referred to as pre-processing. 


The steps need to be taken to do this are the following:
* Tokenizing the document: The process of segmenting a text into words and sentences (i.e: "My name is Jane" ===> ["My", "name", "is", "Jane"])
* Removing the punctuations: After tokenization, there may be punctuations in the list. These need to be removed.
* Removing stopwords: Stopwords include words that are commonly used: "a", "an", "the", "from", etc. These are not useful words for the natural language processing and needs to be filtered out. NLTK library includes a list of stopwords.
* Stemming: In the list, there may be different words with the same word stem. For example, the stem of the words "waited", "waits", "waiting" is "wait". Thus, in the list, the stems of words need to be found. 
* Final cleaning: What is really useful for us is words so any strings including numbers (or anything rather than letters) will be removed.

The code below defines a function named "text_process", and it is used for a document representation.

In [2]:
from nltk.corpus import reuters
from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import re

# TEXT RE-PROCESSING
def text_process(text):
    # Tokenize the text: Separate all elements of a text and put them in a list 
    tokenize_text = word_tokenize(text) 

    # Remove punctuations: In the list we obtained in the previous step, we have punctuations as well. Let's get rid of them
    nopunct_text = [item for item in tokenize_text if item not in string.punctuation]

    # Remove stopWords
    text_noStopWords = [word for word in nopunct_text if word.lower() not in stopwords.words('english')]
    
    # Stem each one of the tokens:
    # Now we only have words as strings in our list but there may be same words in different forms
    # For example, if we have "running", "ran", "run" in the list, instead of different forms of the word, we only want the word "run"
    stem_text = (list(map(lambda token: PorterStemmer().stem(token), text_noStopWords)))

    # Remove any string that contains anything other than letters (ones with numbers will be removed for example)
    regex = re.compile('[a-zA-Z]+')
    clean_tokens = list(filter(lambda token: regex.match(token), stem_text))

    return clean_tokens

# Define Train and Test Files

In Reuters dataset, training and test sets are already splitted. This splitting is necessary to make sure that we do not evaluate the model with the documents that the system is learnt from. In the dataset, documents with names starting with "training" will be used for training of a model. The rest of the documents with name starting with "test" will be used to evaluate the model.

In [3]:
train_docs = []
test_docs = []
train_id = []
test_id = []
 
for id in reuters.fileids():
    if id.startswith("train"):
        train_docs.append(reuters.raw(id))
        train_id.append(id)
    else:
        test_docs.append(reuters.raw(id))
        test_id.append(id)

# Weighting

Our main issue with our data is that it is all in text format (strings). The classification algorithms will need some sort of numerical feature vector in order to perform the classification task. There are many methods to convert a corpus to a vector format. The simplest is the the __bag-of-words__ approach, where each unique word in a text will be represented by one number.

In the large text corpus, there may be some words appear very frequently but carrying little information about the content of the document. If direct counts of tokens are fed into the model, these frequent terms would shadow the rarer but more interesting words. In order to avoid that, the TF-IDF vectorization is be used.

__Term Frequency:__ Importance of the term within that document

TF = Number of times term t appears in a document / Number of terms in the document

__Inverse Term Frequency__ Importance of the term in the corpus

IDF = log(Total number of documents / Number of documents with the term t)

Thus,
__TF-IDF(t, document) = TF(t, document) * IDF(t)__

In order to apply TF-IDF weighting, TfidfVectorizer from sklearn.feature_extraction.text will be used.
Here, fit_transform method is the combination of fit method and the transform method.

__fit():__ Learn vocabulary and idf from training set.

__transform():__ Transform documents to document-term matrix.

Since vocabulary learning is done from the training, fit() method is only called for the training set and not for the test set.

__Note:__ TfidfVectorizer is equivalent to CountVectorizer followed by TfidfTransformer.

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
bow_transformer = TfidfVectorizer(analyzer=text_process)
train_documents = bow_transformer.fit_transform(train_docs)
test_documents = bow_transformer.transform(test_docs)

print(train_documents.shape)

(7769, 21141)


# Categories
In Reuters dataset, there are 90 labels available. One document may be assigned to more than one label (multi-label classification).

MultiLabelBinarizer from sklearn.preprocessing will be used to represent categories of each document in a binary matrix indicating the presence of a class label. For example, if there is 1 instead of the converted category, that means the document under study is labelled with that category. 

In [136]:
from sklearn.preprocessing import MultiLabelBinarizer

# Transform multilabel labels
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform([reuters.categories(id)
                                  for id in train_id])
test_labels = mlb.transform([reuters.categories(id)
                             for id in test_id])

print(reuters.categories())
print(test_labels[60])

[u'acq', u'alum', u'barley', u'bop', u'carcass', u'castor-oil', u'cocoa', u'coconut', u'coconut-oil', u'coffee', u'copper', u'copra-cake', u'corn', u'cotton', u'cotton-oil', u'cpi', u'cpu', u'crude', u'dfl', u'dlr', u'dmk', u'earn', u'fuel', u'gas', u'gnp', u'gold', u'grain', u'groundnut', u'groundnut-oil', u'heat', u'hog', u'housing', u'income', u'instal-debt', u'interest', u'ipi', u'iron-steel', u'jet', u'jobs', u'l-cattle', u'lead', u'lei', u'lin-oil', u'livestock', u'lumber', u'meal-feed', u'money-fx', u'money-supply', u'naphtha', u'nat-gas', u'nickel', u'nkr', u'nzdlr', u'oat', u'oilseed', u'orange', u'palladium', u'palm-oil', u'palmkernel', u'pet-chem', u'platinum', u'potato', u'propane', u'rand', u'rape-oil', u'rapeseed', u'reserves', u'retail', u'rice', u'rubber', u'rye', u'ship', u'silver', u'sorghum', u'soy-meal', u'soy-oil', u'soybean', u'strategic-metal', u'sugar', u'sun-meal', u'sun-oil', u'sunseed', u'tea', u'tin', u'trade', u'veg-oil', u'wheat', u'wpi', u'yen', u'zinc']


# Training a Model
In order to train a model Keras is used. Keras provides utilities and classes for building various kinds of neural networks. The basic workflow consists in 
* loading training data in a numpy array with proper representation, 
* building a model by stacking layers, 
* compiling it,
* finally fitting the model to the data. 

__Sequential:__ Models in Keras are defined as a sequence of layers. We create a Sequential model and add layers one at a time until we are happy with our network topology.
Input layer has the right number of inputs which should be equal to the number of input variables.

__Dense__: Fully connected layers are defined using the Dense class. We can specify the number of neurons in the layer and the activation function using the activation argument.

__Output Layer__: The usual choice for multi-class classification is the softmax layer. A consequence of using the softmax function is that the probability for a class is not independent from the other class probabilies. This is nice as long as we only want to predict a single label per document. By using sigmoid function, the probabilites of each class is independent from the other class probabilities. So we can use the threshold of 0.5 for prediction. 

__Compile__: Once the model looks good, we should configure its learning process with compile() method. An important choice to make is the loss function. It is used to optimize the model. This is the function that will get minimized by the optimizer. We use the binary_crossentropy loss and not the usual in multi-class classification used categorical_crossentropy loss, in order to penalize each output node independently. A metric is used to judge the performance of the model. It doesn't affect the optimization process. Adam is an algorithm for first-order gradient-based optimization of stochastic objective functions, based on adaptive estimates of lower-order moments.

In [123]:
from keras.models import Sequential
model = Sequential()

from keras.layers import Dense
model.add(Dense(units=2000, activation='relu', input_dim=train_documents.shape[1]))
model.add(Dense(units=3000, activation='relu'))
model.add(Dense(units=90, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_documents, train_labels, epochs=4, batch_size=17)

Epoch 1/4
7769/7769 [==============================] - 296s 38ms/step - loss: 0.0371 - acc: 0.9919
Epoch 2/4
7769/7769 [==============================] - 318s 41ms/step - loss: 0.0092 - acc: 0.9971
Epoch 3/4
7769/7769 [==============================] - 307s 39ms/step - loss: 0.0046 - acc: 0.9985
Epoch 4/4
7769/7769 [==============================] - 279s 36ms/step - loss: 0.0029 - acc: 0.9990


In [121]:
score = model.evaluate(test_documents, test_labels, batch_size=17)

3019/3019 [==============================] - 4s 1ms/step


In [122]:
print(score)
model.summary()

[0.015241040769591893, 0.9959625837896865]
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_22 (Dense)             (None, 1000)              21142000  
_________________________________________________________________
dense_23 (Dense)             (None, 3000)              3003000   
_________________________________________________________________
dense_24 (Dense)             (None, 90)                270090    
Total params: 24,415,090
Trainable params: 24,415,090
Non-trainable params: 0
_________________________________________________________________


In [138]:
preds = model.predict(test_documents)

preds[preds>=0.5] = 1
preds[preds<0.5] = 0

pred_new = preds[60]
print(pred_new)
idx = np.where(pred_new>=1)
for val in idx[0]:
    print(reuters.categories()[int(val)])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
earn


In [137]:
# evaluate the model
print("\n%s: %.2f%%" % (model.metrics_names[1], score[1]*100))


acc: 99.60%
